In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import os
import imblearn

In [2]:
main_dir = 'D:/Competitions/datastorm-2021/'


In [3]:
data_dir = main_dir+'data/feature/'
train_file = data_dir+'cb_train.csv'
val_file = data_dir+'cb_validation.csv'
test_file = data_dir+'cb_test.csv'
NN_dir = main_dir+'src/scripts/NeuralNet/'

In [4]:
train_ds = pd.read_csv(train_file)
val_ds = pd.read_csv(val_file)


In [5]:
def start_category_from_zero(df,columns):
    for each in columns:
        df[each] = df[each] - 1

    return df

In [6]:
start_category_from_zero(train_ds,['Reservation_Status'])
start_category_from_zero(val_ds,['Reservation_Status'])

,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,...,Booking_channel,Required_Car_Parking,Use_Promotion,Room_Rate,month_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin,total_participants,Reservation_Status
0,1,56,1,0,0,3,2,2,0,0,...,1,1,0,192,8,2,2,195,4,2
1,1,60,0,2,3,3,2,1,1,0,...,0,0,1,187,8,2,4,175,4,1
2,0,58,3,2,0,0,1,1,0,0,...,2,1,0,227,9,3,1,193,4,1
3,0,23,0,2,3,1,1,1,1,0,...,2,1,0,189,8,2,2,103,3,0
4,0,47,3,2,3,2,0,2,1,0,...,0,0,0,218,8,2,1,92,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744,0,22,1,2,1,2,2,0,1,0,...,0,1,0,199,11,4,3,37,7,0
2745,0,35,1,2,3,2,1,1,0,0,...,2,0,0,106,12,4,1,154,5,1
2746,1,18,3,0,3,0,2,1,0,0,...,1,1,0,136,11,4,2,30,5,2
2747,1,42,3,2,0,3,0,2,1,1,...,0,0,0,226,11,4,1,170,6,0


In [7]:
print(train_ds['Reservation_Status'].value_counts())
train_ds.dtypes

0    21240
1     4134
2     2125
Name: Reservation_Status, dtype: int64


Gender                        int64
Age                           int64
Ethnicity                     int64
Educational_Level             int64
Income                        int64
Country_region                int64
Hotel_Type                    int64
Meal_Type                     int64
Visted_Previously             int64
Previous_Cancellations        int64
Deposit_type                  int64
Booking_channel               int64
Required_Car_Parking          int64
Use_Promotion                 int64
Room_Rate                     int64
month_Expected_checkin        int64
dayofweek_Expected_checkin    int64
stay                          int64
booking_to_checkin            int64
total_participants            int64
Reservation_Status            int64
dtype: object

In [8]:
x_columns = [x for x in train_ds.columns if x != 'Reservation_Status']
y_columns = ['Reservation_Status']
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)
batch_size = 64
input_shape = (len(x_columns))
epochs = 200
print(input_shape)

20


In [9]:
x_columns

['Gender',
 'Age',
 'Ethnicity',
 'Educational_Level',
 'Income',
 'Country_region',
 'Hotel_Type',
 'Meal_Type',
 'Visted_Previously',
 'Previous_Cancellations',
 'Deposit_type',
 'Booking_channel',
 'Required_Car_Parking',
 'Use_Promotion',
 'Room_Rate',
 'month_Expected_checkin',
 'dayofweek_Expected_checkin',
 'stay',
 'booking_to_checkin',
 'total_participants']

In [10]:
x_train = np.asarray(train_ds[x_columns])
y_train = np.asarray(train_ds[y_columns])
x_val = np.asarray(val_ds[x_columns])
y_val = np.asarray(val_ds[y_columns])

In [11]:
x_train.shape

(27499, 20)

In [12]:
y_train.shape

(27499, 1)

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
x_sm, y_train = sm.fit_resample(x_train, y_train)
x_train = pd.DataFrame(x_sm,columns=x_columns)

In [14]:
x_train.shape

(63720, 20)

In [15]:
y_train.shape

(63720,)

In [19]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(64,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.6))
model.add(tf.keras.layers.Dense(64,activation='tanh'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3,activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= optimizer,
    metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data = (x_val,y_val))

In [ ]:
prediction_val = model.predict(x_val)
prediction_train  = model.predict(x_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def func(x):
    return x.argmax()
prediction_train = np.apply_along_axis(func, 1, prediction_train)
prediction_val = np.apply_along_axis(func, 1, prediction_val)

In [ ]:
y_train

In [ ]:
stats =  {
    'train' : {
        'accuracy' : accuracy_score(y_train, prediction_train),
        'precision' : precision_score(y_train,prediction_train, average='macro'),
        'recall' : recall_score(y_train, prediction_train, average='macro'),
        'f1-score' : f1_score(y_train, prediction_train, average='macro')},
    'test' : {
        'accuracy' : accuracy_score(y_val, prediction_val),
        'precision' : precision_score(y_val, prediction_val, average='macro'),
        'recall' : recall_score(y_val, prediction_val, average='macro'),
        'f1-score' : f1_score(y_val, prediction_val, average='macro')
    }
}

In [ ]:
print(stats)
with open(NN_dir+'score_BEFOREencode_cb.json','w') as pf:
    json.dump(stats, pf)

In [ ]:
import matplotlib.pyplot as plt
def acc_n_loss(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    with open("metrics.json", 'w') as outfile:
        json.dump({"Training-accuracy": acc[-1], "Validation-accuracy": val_acc[-1], "Training-loss": loss[-1],
                   "Validation-loss": val_loss[-1]}, outfile)

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
acc_n_loss(history)

In [ ]:
test_ds = pd.read_csv(test_file)

In [ ]:
test_ds

In [ ]:
assert list(test_ds.iloc[:,1:].columns) == x_columns

In [ ]:
x_test = np.asarray(test_ds.iloc[:,1:])

In [ ]:
x_test

In [ ]:
prediction_test = model.predict(x_test)

In [ ]:
prediction_test = np.apply_along_axis(func, 1, prediction_test)

In [ ]:
prediction_test

In [ ]:
sub_df = pd.DataFrame()
sub_df['Reservation-id'] = test_ds.iloc[:, 0]
sub_df['Reservation_status'] = prediction_test